In [ ]:
import pandas as pd

def parse_input(input_str):
    """Parse the input string into a list of floats. Handle both commas and spaces as delimiters."""
    cleaned_input = input_str.replace(',', ' ')  # Replace commas with spaces for uniformity
    try:
        return [float(num) for num in cleaned_input.split()]
    except ValueError:
        raise ValueError("Input contains non-numeric values.")

def confirm_input(data):
    """Display parsed data and ask for user confirmation to proceed."""
    print("You entered:", data)
    return input("Is this correct? (yes/no): ").strip().lower() == 'yes'

def input_and_confirm(prompt, expected_length=None):
    """Handle the input and confirmation process for a given prompt, ensuring data length matches expected if provided."""
    while True:
        user_input = input(prompt)
        try:
            parsed_data = parse_input(user_input)
            if expected_length is not None and len(parsed_data) != expected_length:
                print(f"The number of entries does not match the expected {expected_length}. Please re-enter the data correctly.")
                continue
            if confirm_input(parsed_data):
                return parsed_data
            else:
                print("Let's try entering the data again.")
        except ValueError as e:
            print(str(e))
            print("Please re-enter the data correctly.")

def calculate_heating_duties(ts_list, tt_list, cp_list):
    """Calculate heating duties based on temperatures and heat capacities."""
    return [(tt - ts) * cp for ts, tt, cp in zip(ts_list, tt_list, cp_list)]

def source_data():
    ts_list = input_and_confirm("Enter supply temperature values (separated by spaces or commas): ")
    tt_list = input_and_confirm("Enter target temperature values (separated by spaces or commas): ", len(ts_list))
    cp_list = input_and_confirm("Enter heat capacity values if available (separated by spaces or commas): ", len(ts_list))
    q_values = input("Enter values for heat duty if available (separated by spaces or commas, optional): ")
    q_list = parse_input(q_values) if q_values.strip() else []

    if not q_list:
        q_list = calculate_heating_duties(ts_list, tt_list, cp_list)

    label_list = ['hot' if ts > tt else 'cold' for ts, tt in zip(ts_list, tt_list)]
    input_data = {'Tsupply': ts_list, 'Ttarget': tt_list, 'Cp': cp_list, 'Q': q_list}
    df = pd.DataFrame(input_data, index=label_list)

    print("Final confirmed data:")
    print(df)
    return df

# Uncomment the following line to execute in an actual environment
# df = source_data()

In [ ]:
process_data = source_data()

Enter supply temperature values (separated by spaces or commas): 24 56 43 24 65
You entered: [24.0, 56.0, 43.0, 24.0, 65.0]
Is this correct? (yes/no): yes
Enter target temperature values (separated by spaces or commas): 34, 56, 54, 64 90, 56
The number of entries does not match the expected 5. Please re-enter the data correctly.
Enter target temperature values (separated by spaces or commas): 34, 56, 54, 56 78
You entered: [34.0, 56.0, 54.0, 56.0, 78.0]
Is this correct? (yes/no): yes
Enter heat capacity values if available (separated by spaces or commas): 
The number of entries does not match the expected 5. Please re-enter the data correctly.
Enter heat capacity values if available (separated by spaces or commas): 2.1 4.3 5.2 1.1 2.3
You entered: [2.1, 4.3, 5.2, 1.1, 2.3]
Is this correct? (yes/no): yes
Enter values for heat duty if available (separated by spaces or commas, optional): 
Final confirmed data:
      Tsupply  Ttarget   Cp     Q
cold     24.0     34.0  2.1  21.0
cold     56

In [ ]:
process_data

,Tsupply,Ttarget,Cp,Q
cold,24.0,34.0,2.1,21.0
cold,56.0,56.0,4.3,0.0
cold,43.0,54.0,5.2,57.2
cold,24.0,56.0,1.1,35.2
cold,65.0,78.0,2.3,29.9


In [ ]:
def create_problem_table(df):
    """
    Create a problem table for pinch analysis from a given DataFrame.
    Assumes the DataFrame has columns: 'Tsupply', 'Ttarget', 'Cp', and 'Q'.
    """
    # Ensure the DataFrame is sorted by Tsupply and Ttarget appropriately
    df = df.sort_values(by=['Tsupply', 'Ttarget'], ascending=[False, True])

    # Extract all unique temperatures and sort them in descending order
    temperatures = sorted(set(df['Tsupply']).union(df['Ttarget']), reverse=True)

    # Initialize the problem table with temperature intervals
    problem_table = pd.DataFrame({'Temp': temperatures[:-1]})

    # Calculate heat flows for each temperature interval
    heat_flows = []
    for i in range(len(temperatures) - 1):
        upper_temp = temperatures[i]
        lower_temp = temperatures[i + 1]

        # Sum heat duties for hot streams cooling down through this interval
        q_hot = sum(df[(df['Tsupply'] > upper_temp) & (df['Ttarget'] < upper_temp)]['Q'])
        # Sum heat duties for cold streams heating up through this interval
        q_cold = sum(df[(df['Tsupply'] <= lower_temp) & (df['Ttarget'] >= lower_temp)]['Q'])

        # The heat flow for an interval is the heat required by cold streams minus the heat supplied by hot streams
        heat_flow = q_cold - q_hot
        heat_flows.append(heat_flow)

    problem_table['Heat Flow'] = heat_flows

    # Calculate the cumulative heat flow to identify the pinch point
    problem_table['Cumulative Heat Flow'] = problem_table['Heat Flow'].cumsum()

    return problem_table



In [ ]:
problem_table = create_problem_table(process_data)
problem_table

,Temp,Heat Flow,Cumulative Heat Flow
0,78.0,29.9,29.9
1,65.0,35.2,65.1
2,56.0,92.4,157.5
3,54.0,92.4,249.9
4,43.0,56.2,306.1
5,34.0,56.2,362.3


In [ ]:
def create_problem_table_with_shift(df, delta_tmin):
    """
    Create a problem table for pinch analysis from a given DataFrame with temperature shifting.

    Args:
    df (DataFrame): The input data containing 'Tsupply', 'Ttarget', 'Cp', and 'Q'.
    delta_tmin (float): The minimum temperature difference for heat exchange between streams.

    Returns:
    DataFrame: The problem table with temperature intervals, heat flows, and cumulative heat flows.
    """
    # Apply temperature shifts
    df_shifted = df.copy()
    df_shifted.loc[df_shifted['Q'] > 0, 'Tsupply'] += delta_tmin / 2  # Cold streams
    df_shifted.loc[df_shifted['Q'] > 0, 'Ttarget'] += delta_tmin / 2
    df_shifted.loc[df_shifted['Q'] < 0, 'Tsupply'] -= delta_tmin / 2  # Hot streams
    df_shifted.loc[df_shifted['Q'] < 0, 'Ttarget'] -= delta_tmin / 2

    # Create combined temperature list and define intervals
    temperatures = sorted(set(df_shifted['Tsupply']).union(df_shifted['Ttarget']), reverse=True)

    # Initialize problem table with temperature intervals
    problem_table = pd.DataFrame({'Temp': temperatures[:-1]})

    # Calculate heat flows for each temperature interval
    heat_flows = []
    for i in range(len(temperatures) - 1):
        upper_temp = temperatures[i]
        lower_temp = temperatures[i + 1]

        # Calculate the heat flow for the interval using the shifted temperatures
        q_hot = sum(df_shifted[(df_shifted['Tsupply'] > upper_temp) & (df_shifted['Ttarget'] < upper_temp)]['Q'])
        q_cold = sum(df_shifted[(df_shifted['Tsupply'] <= lower_temp) & (df_shifted['Ttarget'] >= lower_temp)]['Q'])

        # The heat flow for an interval is the heat required by cold streams minus the heat supplied by hot streams
        heat_flow = q_cold - q_hot
        heat_flows.append(heat_flow)

    problem_table['Heat Flow'] = heat_flows
    problem_table['Cumulative Heat Flow'] = problem_table['Heat Flow'].cumsum()

    return problem_table

# Example usage with a specified Delta Tmin
# Uncomment the line below to use the function with an actual DataFrame `df` and Delta Tmin `delta_tmin`.
# problem_table_with_shift = create_problem_table_with_shift(df, delta_tmin)
